In [ ]:
import warnings
warnings.filterwarnings('ignore', module="h5py")

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import numpy as np
import itertools
import collections
from types import SimpleNamespace
from mpnetcdf4.utils import Timer, norm_selection, interleave_n, shape_from_slice
from mpnetcdf4.benchmarking import (
    with_stats,
    run_benchmark,
    dump_as_pickle,
    plot_benchmark_results)

from mpnetcdf4 import ReaderFactory
from mpnetcdf4.ncread import VStackReader, alloc_empty

def load_file_list(fname):
    with open(fname, 'rt') as f:
        return [s.rstrip() for s in f.readlines()]

In [ ]:
params = SimpleNamespace(
    mb=100,
    nprocs=8,
    xy_roi=np.s_[:,:], #1000:1800, 1000:1800],
    chunk_scale=(1,2,2),
    measurements=('red','green', 'blue', 'nir', 'swir1', 'swir2', 'coastal_aerosol')
)

mpr = ReaderFactory(params.nprocs, mb=params.mb)

In [ ]:
ff = load_file_list('ls8_tile_2016.txt')[:(8*2)]
len(ff), ff[:3]

In [ ]:
rdr = VStackReader(ff, mpr, params)
ds = rdr.alloc(params.xy_roi)

### warm up

In [ ]:
ds, stats = with_stats(rdr.read)(params.xy_roi, ds)
stats

In [ ]:
ds.red.isel(time=11).plot.imshow(add_colorbar=False,
                                 add_labels=False,
                                 );

In [ ]:
plt.plot(ds.red.values[:, 100, 101], '.')

### run benchmark

In [ ]:
all_stats = []

def run_bench(xy_roi, ds, nprocs):
    from copy import copy
    
    pp = copy(params)
    pp.nprocs = nprocs
    pp.xy_roi = xy_roi
    
    print('Starting read: %d procs' % (nprocs))
    
    ds, stats = with_stats(rdr.read)(xy_roi, ds, nprocs=nprocs)
    
    print('Read %d mb in %.3f secs (%.3f Mb/s, %.3f Mb/s/proc)' %(
        stats.mb, stats.elapsed,
        stats.throughput_mb,
        stats.throughput_mb/nprocs
    ))
    
    stats.t_total = stats.elapsed
    stats.params = pp
    
    return ds, stats

for nprocs in range(1, params.nprocs+1):
    ds, stats = run_bench(params.xy_roi, ds, nprocs=nprocs)
    all_stats.append(stats)

In [ ]:
plot_benchmark_results(all_stats, fig=plt.figure(figsize=(16,4)))